In [1]:
## Rewrite to add TI indicators to symbols from machine_learning_dict

In [3]:
import pandas as pd
import numpy as np
# import requests
import json
# import hvplot.pandas
# from dotenv import load_dotenv
from datetime import date
import os
from scipy import stats
from numpy.lib.stride_tricks import as_strided
from numpy.lib import pad
#import pad
# import matplotlib.pyplot as plt
# %matplotlib inline

from pathlib import Path

In [4]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [29]:
def rolling_spearman(seqa, seqb, window):
    stridea = seqa.values.strides[0]
    ssa = as_strided(seqa, shape=[len(seqa) - window + 1, window], strides=[stridea, stridea])
    strideb = seqa.values.strides[0]
    ssb = as_strided(seqb, shape=[len(seqb) - window + 1, window], strides =[strideb, strideb])
    ar = pd.DataFrame(ssa)
    br = pd.DataFrame(ssb)
    ar = ar.rank(1)
    br = br.rank(1)
    corrs = ar.corrwith(br, 1)
    return pad(corrs, (window - 1, 0), 'constant', constant_values=np.nan)

# def create_price_df(self,ticker, period='daily'):

#     if period =='hourly':
#         api_url = 'https://fmpcloud.io/api/v3/historical-chart/1hour'
#     else:
#         api_url = 'https://fmpcloud.io/api/v3/historical-price-full'

#     ticker_df = json.loads(requests.get(f"{api_url}/{ticker}?apikey={fmp_api}").content)['historical']
#     data = pd.DataFrame(ticker_df).set_index('date')[::-1]
#     data['Date'] = data.index
#     data.index = data.index.astype('datetime64[ns]')

#     return data

def use_csvs(ticker):

    #data = pd.read_csv("../FilesExportIndividualStockDFs_Big/"+ticker+"_combined_df.csv", index_col='Date', parse_dates=True)
    path = Path('../FilesExport_Complete_DataDicts/'+ticker+'_data_dict.pkl')
    data_import = load_obj(path)
    data = data_import['dataFrame'].copy()

    return data

def get_dataframe(symbol):

    #data = pd.read_csv("../FilesExportIndividualStockDFs_Big/"+ticker+"_combined_df.csv", index_col='Date', parse_dates=True)
    path = Path('../FilesExport_Complete_DataDicts/'+symbol+'_data_dict.pkl')
    data_import = load_obj(path)
    data = data_import[symbol]['dataFrame']

    return data



def bollinger_bands(dataframe,period=20):
    data = dataframe.copy()
    data['middle_band'] = data[['adjClose']].rolling(window=period).mean()
    data[str(period)+'_day_stdev'] = data[['adjClose']].rolling(window=period).std()
    data['upper_band'] = data['middle_band']+2*data[str(period)+'_day_stdev']
    data['lower_band'] = data['middle_band'] - 2*data[str(period)+'_day_stdev']
    data['spread'] = data['upper_band'] + data['lower_band']
    data['change_in_spread'] = data['spread']/data['spread'].shift(1)-1
    data[str(period)+"_return"] = data['adjClose']/data['adjClose'].shift(period)-1
    data['bollinger_signal'] = data['change_in_spread'].rank(ascending=False, pct=True)
    data.dropna()

    return data

def dema(dataframe, period1=10, period2=20):
    data = dataframe.copy()
    data[str(period1)+'ema1'] = dataframe[['adjClose']].ewm(span=period1, adjust=False).mean()
    data[str(period1)+'ema2'] = data[str(period1)+'ema1'].ewm(span=period1, adjust=False).mean()
    data[str(period1)+'dema'] = 2*data[str(period1)+'ema1'] - data[str(period1)+'ema2']
    data[str(period2)+'ema1'] = data[['adjClose']].ewm(span=period2, adjust=False).mean()
    data[str(period2)+'ema2'] = data[str(period2)+'ema1'].ewm(span=period2, adjust=False).mean()
    data[str(period2)+'dema'] = 2*data[str(period2)+'ema1'] - data[str(period2)+'ema2']
    data[str(period1)+"_return"] = data['adjClose']/data['adjClose'].shift(period1)-1
    data['spread'] = data[str(period1)+'dema'] - data[str(period2)+'dema']
    data['dema_signal'] = data['spread'].rank(ascending=True, pct=True)
    data = data.dropna()
    return data

def price_momentum(dataframe, smoothing1=0.0571, smoothing2=0.1, periods1=15, periods2=10):
    data = dataframe.copy()
    data['smoothing_factor'] = smoothing1
    data[str(periods1)+"average"] = data['changeOverTime'].rolling(window=periods1).mean()
    smoothing_factor_list = [data.iloc[periods1][str(periods1)+"average"]]
    data = data.dropna()
    i=1
    j=0
    while i < len(data[str(periods1)+"average"]):
        smoothing_factor = data.iloc[i]['changeOverTime']*data.iloc[i]['smoothing_factor'] + smoothing_factor_list[j]*(1-data.iloc[i]['smoothing_factor'])
        smoothing_factor_list.append(smoothing_factor)
        j+=1
        i+=1
    data['35d_custom_smoothing'] = smoothing_factor_list
    data['35d_custom_10'] = data['35d_custom_smoothing']*10
    data['smoothing_factor2'] = smoothing2
    data[str(periods2)+"average"] = data['35d_custom_10'].rolling(window=periods2).mean()
    data = data.dropna()
    smoothing_factor_list2 = [data.iloc[0][str(periods2)+"average"]]
    i=1
    j=0
    while i < len(data[str(periods2)+"average"]):
        smoothing_factor = (data.iloc[i]['35d_custom_10'] - smoothing_factor_list2[j])*data.iloc[i]['smoothing_factor2'] + smoothing_factor_list2[j]
        smoothing_factor_list2.append(smoothing_factor)
        j+=1
        i+=1
    data[str(periods2)+'d_custom_smoothing'] = smoothing_factor_list2
    data[str(periods2)+"_return"] = data['adjClose']/data['adjClose'].shift(periods2)-1
#         data['signal'] = np.where(data[str(periods2)+'d_custom_smoothing'] > data[str(periods2)+'d_custom_smoothing'].shift(1), 1.0, 0.0)
#         data = data.rename(columns={'signal':'price_mo'})
    return data

def get_ichimoku_cloud(dataframe, period1=4, period2=8, period3=15):

    #TODO generate signal, ichimoku works better in current market regime with shorter periods, being able to respond faster to events than a traditional version
    # The conversion crossing the base would be the signal

    data = dataframe.copy()
    data['conversion_line'] = data[['adjClose']].rolling(window=period1).mean()
    data['base_line'] = data[['adjClose']].rolling(window=period2).mean()
    data['senkou_spanA_line'] = (data['conversion_line']+data['base_line'])/2
    data['senkou_spanB_line'] = data[['adjClose']].rolling(window=period3).mean()
    data['lagging_span'] = data['adjClose'].shift(period2)
    data = data.dropna()

    return data

def accumulation_distribution_line(dataframe):

    ##TODO define periodicity and pass as arguments, use the mean as the signal generator, -1 is buy and and 1 is sell
    ##TODO need to add ability to ignore a -1 during a range of 1s

    data = dataframe.copy()
    data['money_flow_mult'] = round(((data['adjClose'] - data['low']) - (data['high'] - data['adjClose']))/(data['high'] - data['low']),2)
    data = data.dropna()
    data['money_flow_volume'] = data['money_flow_mult']*data['volume']
    money_flow_multiplier_list = list(data['money_flow_volume'].values)
    adl = [money_flow_multiplier_list[0]]
    i = 1
    while i < len(money_flow_multiplier_list):
        a_d_indicator = adl[i-1]+money_flow_multiplier_list[i]
        adl.append(a_d_indicator)
        i+=1
    data['adl'] = adl
    data['adl_change'] = data['adl']/data['adl'].shift(1)-1
    negative_change_count = [0]*9
    i = 0
    counter = 0
    while i < len(data)-9:
        j=0
        while j < 9:
            if data.iloc[j+i]['adl_change'] <0:
                counter+=1
            if j %19 == 0:
                negative_change_count.append(counter)
                counter = 0
            j+=1
        i+=1
    data['negative_change_counter'] = negative_change_count
    data['9_day_return'] = data['adjClose']/data['adjClose'].shift(9)-1
    data['adl_signal'] = rolling_spearman(data['adl'], data['9_day_return'], 9)

    return data

def rsi(dataframe, periods=14):
    data = dataframe.copy()
    data['gains'] = np.where(data['changeOverTime']>0, data['changeOverTime'], 0)
    data['losses'] = np.where(data['changeOverTime']<0, np.absolute(data['changeOverTime']), 0)
    data['average_gain'] = data['gains'].rolling(window=periods).mean()
    data['average_loss'] = data['losses'].rolling(window=periods).mean()
    data['rs'] = data['average_gain']/data['average_loss']
    data['rsi'] = (100 - 100/(1+data['rs']))
    data['rsi_signal'] = data['rsi'].rank(ascending=True, pct=True)

    return data


# def get_ratings(self):

#     ratings = json.loads(requests.get(f"https://fmpcloud.io/api/v3/historical-rating/{self.ticker}?limit=100&apikey={fmp_api}").content)
#     ratings_df = pd.DataFrame(ratings)
#     ratings_df['average_rating'] = (ratings_df['ratingScore']+ratings_df['ratingDetailsDCFScore']+ratings_df['ratingDetailsROEScore']+ratings_df['ratingDetailsROAScore'] \
#                                     +ratings_df['ratingDetailsDEScore'] + ratings_df['ratingDetailsPEScore']+ ratings_df['ratingDetailsPBScore'])/7

#     return ratings_df

# def get_stock_market_performances(self, dataframe):
#     limit = len(dataframe)
#     data = json.loads(requests.get(f"https://fmpcloud.io/api/v3/historical-sectors-performance?limit=525&apikey={fmp_api}").content)
#     sector_df = pd.DataFrame(data).set_index('date')
#     sector_df_clean = sector_df[::-1]
#     s_p500 = json.loads(requests.get(f"https://fmpcloud.io/api/v3/historical-price-full/^SP500TR?from="+sector_df_clean.index[0]+"&to="+sector_df_clean.index[-1]+"&apikey="+fmp_api).content)['historical']
#     sp_df = pd.DataFrame(s_p500)
#     sp_df_clean = sp_df[::-1]

#     return sp_df_clean

def get_all_indicators(symbol):
    dataframe = get_dataframe(symbol)
    bb = bollinger_bands(dataframe)
    DEMA = dema(dataframe)
    ADL = accumulation_distribution_line(dataframe)
    RSI = rsi(dataframe)

    dataframe['bollinger_signal'] = bb['bollinger_signal']
    dataframe['dema_signal'] = DEMA['dema_signal']
    dataframe['adl_signal'] = ADL['adl_signal']
    dataframe['rsi_signal'] = RSI['rsi_signal']
    return dataframe

# def merge_data(self):
#     ticker_list = self.stock_list
#     for stock in ticker_list:
#         stock_csv = self.use_csvs(stock)
#         stock_csv['Date'] = stock_csv.index.astype("string")
#         stock_csv = stock_csv.iloc[:-1]
#         stock_csv.index.names =[""]
#         stock_indicators = self.get_all_indicators(stock)
#         stock_signals = stock_indicators[['Date','bollinger_signal','dema_signal', 'adl_signal', 'rsi_signal']]
#         stock_signals.loc[:,'Date'] = stock_signals['Date'].astype('string')
#         stock_signals.index.names =[""]
#         merged_data = pd.merge(left = stock_csv, right=stock_signals, on=['Date']).set_index('Date')
#         merged_data = merged_data.dropna()
#         writer = pd.ExcelWriter("NewCsvs/"+stock+"_all_indicator_dfs.xlsx", engine="xlsxwriter")

#         merged_data.to_excel(writer, sheet_name=stock+"_data")

#         writer.save()

#     return print("All files successfully saved")

In [30]:
# symbol = 'GME'
# path = Path('../FilesExport_Complete_DataDicts/'+symbol+'_data_dict.pkl')
# data_import = load_obj(path)
# data_import[symbol]['dataFrame']

In [31]:
test_df = get_all_indicators('XRT')

In [32]:
test_df.dropna()

,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,...,ShortVolumeNSDQ,ShortExemptVolumeNSDQ,TotalVolumeNSDQ,ShortVolumeNYSE,ShortExemptVolumeNYSE,TotalVolumeNYSE,bollinger_signal,dema_signal,adl_signal,rsi_signal
Date,,,,,,,,,,,,,,,,,,,,,
2016-02-02,41.180000,41.320000,40.279999,40.430000,37.394249,5099200.0,5099200.0,-0.75,-1.821,40.67667,...,949497.0,4100.0,1145118.0,78308.0,0.0,88224.0,0.893432,0.755952,-0.883333,0.312208
2016-02-03,40.419998,40.680000,39.419998,40.169998,37.153763,5722400.0,5722400.0,-0.25,-0.619,40.09000,...,940915.0,5.0,1240792.0,123426.0,0.0,139891.0,0.923592,0.727513,-0.783333,0.507005
2016-02-04,39.540001,40.400002,39.400002,40.169998,37.153763,6053300.0,6053300.0,0.63,1.593,39.99000,...,868617.0,18207.0,1019075.0,64048.0,0.0,80169.0,0.874665,0.708333,-0.566667,0.634423
2016-02-05,39.980000,40.099998,39.040001,39.110001,36.173367,4187600.0,4187600.0,-0.87,-2.176,39.41667,...,596052.0,1208.0,723741.0,34035.0,0.0,42558.0,0.924263,0.589286,-0.100000,0.456971
2016-02-08,38.770000,38.880001,38.099998,38.650002,35.747902,4747300.0,4747300.0,-0.12,-0.310,38.54333,...,587575.0,6488.0,1189653.0,44016.0,0.0,54324.0,0.882038,0.447751,0.500000,0.556371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-27,88.830000,89.890000,88.430000,89.760000,89.760000,2525984.0,2525984.0,0.93,1.047,89.36000,...,233244.0,1399.0,502637.0,17595.0,0.0,29841.0,0.930295,0.306217,-0.100000,0.308873
2021-12-28,89.510000,90.700000,89.235000,89.330000,89.330000,2066502.0,2066502.0,-0.18,-0.201,89.75500,...,104998.0,84.0,230222.0,8570.0,100.0,84262.0,0.929625,0.601190,0.366667,0.245497
2021-12-29,89.450000,90.225000,89.010000,89.850000,89.850000,2416159.0,2416159.0,0.40,0.447,89.69500,...,263015.0,3462.0,399182.0,8401.0,0.0,14118.0,0.843834,0.814153,0.683333,0.334223


In [33]:
import_path = '../Resources/05_machine_learning_dict.pkl'
machine_learning_dict = load_obj(import_path)
len(machine_learning_dict)

1563

In [34]:
## put symbols in a list, for testing 
symbol_list = []
for symbol in machine_learning_dict:
    symbol_list.append(symbol)

In [35]:
test_list = symbol_list[0:10]
test_list

['AAA', 'AACG', 'AAL', 'AAME', 'AAU', 'AAXJ', 'ABEO', 'ABIO', 'ABUS', 'ACB']

In [38]:
data_dict = {} 
symbol_success_list = []
for symbol in test_list:
    temp_dict = {}
    try:
        df = get_all_indicators(symbol)
    except: continue 
        
    df = df.dropna()
    
    if len(df) > 1400:
        temp_dict[symbol] = df
        data_dict[symbol] = df 
        symbol_success_list.append(symbol)
        
        ## Export Temp dict 
        
len(data_dict) ## If successful, should be 10

6

In [41]:
data_dict

{'AAL':                  open       high        low      close   adjClose      volume  \
 Date                                                                            
 2016-02-02  38.830002  38.970001  37.000000  37.029999  35.387558  12401400.0   
 2016-02-03  37.380001  37.610001  35.759998  37.509998  35.846268  12390500.0   
 2016-02-04  37.340000  38.599998  36.700001  38.209999  36.515217   8599600.0   
 2016-02-05  37.709999  38.490002  36.529999  36.750000  35.119984  10072700.0   
 2016-02-08  36.080002  36.160000  34.759998  35.549999  34.065891  12821900.0   
 ...               ...        ...        ...        ...        ...         ...   
 2021-12-27  17.660000  18.240000  17.545000  18.170000  18.170000  27835600.0   
 2021-12-28  18.030000  18.640000  17.930000  18.540000  18.540000  28057128.0   
 2021-12-29  18.400000  18.430000  17.990000  18.050000  18.050000  22521456.0   
 2021-12-30  17.980000  18.380000  17.960000  18.070000  18.070000  28054659.0   
 2021-12-